<a href="https://colab.research.google.com/github/prushton2/977Z/blob/master/RoboticsScoreProgram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Google Drive


In [255]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Class definitions

## Json manager class

In [0]:
import matplotlib.pyplot as plt
import csv
import json
from termcolor import colored, cprint 
import time 

null = None #I can't be bothered to remember null is None in python
true = True
false = False

def empty(): #Exists to do nothing for the sake of blank constructors or unfinished methods
  return None;

class JsonManager:
  def __init__(self, path, fType): # fType should be 'score', 'comp', or 'team' depending on which json file is being opened
    self.path = path
    self.fType = fType
  def load(self):
    #opens json file and returns data
    with open(self.path, mode="r") as theFile:
      return json.load(theFile) 
      
  def save(self, data):
    #opens json file and saves data
    with open(self.path, mode="w") as theFile:
      json.dump(data, theFile)

  def addData(self, key, value, function): #Adds data to a json key, but abstracted. 
    jsonFile = self.load()#JsonManager.open()

    if(function == 'set'):
      exec("jsonFile"+str(key)+" = "+str(value))
    elif(function == 'append'):
      exec("jsonFile"+str(key)+".append("+str(value)+")")
    else:
      raise ValueError("The function parameter must be 'set' or 'append', not", function)
    self.save(jsonFile)

  

## Competition Manager

In [0]:
class Comp: #This is used as a datatype to hold the data for a competition
  def __init__(self, id, name, date, path):
    self.id = id;
    self.name = name;
    self.date = date;
    self.path = path;

class AllComps:
  def __init__(self, compList):
    self.compList = compList;
  def getCompFromId(self, id):
    for i in self.compList:
      if(i.id == id):
        return i

## Compiler

In [0]:
class Compiler:
  def __init__(self, comps):
    self.comps = comps;

  def setCompIDs(self, comps):
    self.comps = comps
  
  def compileComps(self): # Will compile the csv data into the jsons.
    for thisComp in self.comps:
      compFile.addData('['+str(thisComp.id)+']', {"Matches":[],"Skills":[]}, 'set')

      with open(thisComp.path, mode='r') as thisFile:
        reader = csv.DictReader(thisFile)
        line_count=0
        for row in reader:
          print("Adding match", row["Match"], "of comp", thisComp.name)
          if(int(row["Red Score"]) > int(row["Blue Score"])):
            winner = 'Red'
          elif(int(row["Blue Score"]) > int(row["Red Score"])):
            winner = 'Blue'
          else:
            winner = null
          thisMatch = [
            row["Match"],
            thisComp.id, 
            [int(row["Red Score"]), int(row["Blue Score"])],
            [row["Red Team (1)"], row["Red Team (2)"]], 
            [row["Blue Team (1)"], row["Blue Team (2)"]],
            winner
          ]

          compFile.addData('["'+str(thisComp.id)+'"]["Matches"]', thisMatch, 'append')
  
  def compileTeams(self):
    json = compFile.load()
    for i in json:
      for j in json[i]["Matches"]:
        for k in j[3]:
          try:
            teamFile.addData("['"+k+"']['Matches']", j, 'append')
            print("Added", j, "to team", k)
          except:
            teamFile.addData("['"+k+"']", {"Matches":[j],"Skills":[]}, 'set')
            print("Created team", k, "with match", j)
        for k in j[4]:
          try:
            teamFile.addData("['"+k+"']['Matches']", j, 'append')
            print("Added", j, "to team", k)
          except:
            teamFile.addData("['"+k+"']", {"Matches":[j],"Skills":[]}, 'set')
            print("Created team", k, "with match", j)

  def compileScores(self):
    scores = scoreFile.load();
    teams = teamFile.load();
    for team in teams:
      scoreFile.addData("['"+team+"']", [0], 'set')
      print("Created a record for team", team, "with 0 points")
      for match in teams[team]["Matches"]:
        scores = scoreFile.load();
        if(match[5] == 'Red'): #Create the number to add/subtract to the players score
          delta = ((int(match[2][0]) + int(match[2][1])) / int(match[2][0]))*100
        if(match[5] == 'Blue'):
          delta = ((int(match[2][0]) + int(match[2][1])) / int(match[2][1]))*100
        if(match[5] == null):
          delta = 0
        
        if(not(MatchHandler.hasWon(team, match))):
          delta *= -1;

        # print(team)
        z = (scores[team][len(scores[team])-1] + delta)
        scoreFile.addData('["'+team+'"]', z, 'append')
        print("Added", delta, "points to the record for team", team)

          
        


## Match Handler

In [0]:
class MatchHandler:
  @staticmethod
      
  def getColor(team, match):
    if(team == match[3][0] or team == match[3][1]):
      return "Red"
    if(team == match[4][0] or team == match[4][1]):
      return "Blue"

  def hasWon(team, match):
    if(MatchHandler.getColor(team, match) == match[5]):
      return True
    else:
      return False # set cube stack

## Team Class

In [0]:
class Team:
  def __init__(self, teamName):
    self.team = teamName;

  def hasCategory(self, category, categories):
    return category in categories or "all" == categories

  def showStats(self, categories):
    if(self.hasCategory("matches", categories)):
      teams = teamFile.load();
      for i in teams[self.team]["Matches"]:
        print(allComps.getCompFromId(i[1]).name, "on", allComps.getCompFromId(i[1]).date, "   |   ", end="")
        
        if(MatchHandler.hasWon(self.team, i)):
          color='green'
        elif(i[5] == null):
          color='grey'
        else:
          color='red'
        
        print(colored(i[0], color, attrs=['reverse', 'blink']), end='')
        print("  |  ", end='')

        if(i[5] == null):
          print(colored(i[3][0]+", "+i[3][1]+" : "+str(i[2][0])+" vs "+str(i[2][1])+" : "+i[4][0]+", "+i[4][1], 'grey'), end='')

        if(i[5] == 'Red'):
          print(colored(i[3][0]+", "+i[3][1]+" : "+str(i[2][0])+" vs ", 'red'), end='')
          print(str(i[2][1])+" : "+i[4][0]+", "+i[4][1], end='')

        if(i[5] == 'Blue'):
          print(i[3][0]+", "+i[3][1]+" : "+str(i[2][0]), end='')
          print(colored(" vs "+str(i[2][1])+" : "+i[4][0]+", "+i[4][1], 'blue'), end='')



        print("")
        
        
          

## Score Class

In [0]:
class Score:
  def __init__(self, teams):
    self.teams = teams;
  
  def rankTeams():
    empty()


# Instantiations

In [0]:
localpath = "/content/drive/My Drive/RoboticsScoreProgram/Competitions"
compList = [
            Comp(0.0,  "Hartford",                        "10/19/19", localpath+"/VRC_10 19 19_Hartford.csv"),
            Comp(1.0,  "Quinsigamond Community College",  "11/02/19", localpath+"/VRC_11 02 19_Madison.csv"),
            Comp(2.0,  "Madison",                         "11/02/19", localpath+"/VRC_11 02 19_QCC.csv"),
            Comp(3.0,  "Framingham",                      "11/09/19", localpath+"/VRC_11 09 19_Framingham.csv"),
            Comp(4.0,  "Danbury",                         "11/16/19", localpath+"/VRC_11 16 19_Monroe.csv"),
            Comp(5.0,  "North Andover",                   "11/23/19", localpath+"/VRC_11 23 19_NorthAndover.csv"),
            Comp(6.0,  "Hopkinton",                       "12/07/19", localpath+"/VRC_12 07 19_Hopkinton.csv"),
            Comp(7.0,  "CTEEA Masuk",                     "12/07/19", localpath+"/VRC_12 07 19_Masuk.csv"),
            Comp(8.0,  "Quinsigamond Community College",  "12/21/19", localpath+"/VRC_12 21 19_QCC.csv"),
            Comp(9.0,  "CTEEA Bolton CT",                 "01/04/20", localpath+"/VRC_01 04 20_CTEEABolton.csv"),
            Comp(10.0, "Quinsigamond Community College",  "01/04/20", localpath+"/VRC_01 04 20_QCC.csv")
]

allComps = AllComps(compList)
compiler = Compiler(compList)

compFile = JsonManager("/content/drive/My Drive/RoboticsScoreProgram/Json/Comps.json", 'comp')
teamFile = JsonManager("/content/drive/My Drive/RoboticsScoreProgram/Json/Teams.json", 'team')
scoreFile = JsonManager("/content/drive/My Drive/RoboticsScoreProgram/Json/Scores.json", 'score')

# Compiling
Compiles the CSV files into the JSON files

In [0]:
if(false):
  startTime = time.time()
  compFile.save({})
  teamFile.save({})
  scoreFile.save({})

  compiler.compileComps()
  compiler.compileTeams()
  compiler.compileScores()
  print("Operation took", time.time()-startTime,"seconds")


# Debug

In [264]:
print(scores["1898A"][len(scores["1898A"])-1])

147.36842105263156


# Show a team's scores

In [265]:
a977Z = Team("977Z")
a977Z.showStats("all")

Framingham on 11/09/19    |   Qualifier #8  |  10102A, 2442E : 4 vs 36 : 977Z, 63857A
Framingham on 11/09/19    |   Qualifier #15  |  977Z, 4716A : 18 vs 24 : 9228B, 4716D
Framingham on 11/09/19    |   Qualifier #19  |  977Z, 9228A : 15 vs 10 : 3379A, 1898M
Framingham on 11/09/19    |   Qualifier #28  |  48425A, 2442A : 23 vs 40 : 977Z, 1898R
Framingham on 11/09/19    |   Qualifier #42  |  785Z, 64345A : 5 vs 12 : 977Z, 785E
Framingham on 11/09/19    |   Qualifier #52  |  977Z, 63857D : 30 vs 12 : 9421R, 4344X
Framingham on 11/09/19    |   Qualifier #56  |  10102X, 977Z : 12 vs 3 : 9421V, 2713B
Framingham on 11/09/19    |   Qualifier #66  |  2713A, 9421X : 27 vs 34 : 977Z, 1898A
Framingham on 11/09/19    |   R16 #3-1  |  1898A, 977Z : 39 vs 15 : 10102X, 10102A
Framingham on 11/09/19    |   QF #2-1  |  1898A, 977Z : 19 vs 3 : 63857E, 9421X
Framingham on 11/09/19    |   SF #1-1  |  1898R, 375X : 28 vs 20 : 1898A, 977Z
North Andover on 11/23/19    |   Qualifier #9  |  977Z, 4478D : 30 vs 

# Structure
* Json Files
  * Competitions:
    * Holds every played match sorted by competition then time
  * Scores
    * Holds every team's score, and their score over time stored in a list
  * Teams
    * Stores every team, and every match they have played.

# Backlog

### Complete
* Create Json manager class
* Create competition class
* Create competition ID system
* Read and compile csvs into jsons
  * Have a json for comps and a json for teams
* Create scoring system

### In progress
* Print a teams stats nicely
* Show scores over time

### TODO

# Sprint Task List

#### Complete

#### In progress
* Iterate through team matches and print nicely
* Create class to hold team name

#### TODO

# Json format:

## Comps:
    {
      "CompName":{
        "Matches":[
          [
            "matchName",
            compId,
            [redScore, blueScore],
            [red1, red2],
            [blue1, blue2],
            "winner"
          ]
        ],
        "Skills": [
          [
            "TeamName",
            [autonScore, driverScore]
          ]
        ]
      }
    }
## Scores:

    {
      "teamNumber": [
        score1,
        score2,
        score3...
      ]
    }

## Teams:

    {
      "teamNumber": {
        "Matches: [
          "referenceToCompsJsonMatchIndex",
        ],
        "Skills": [

        ]
      }
    }


